In [0]:
from io import BytesIO
from json import loads
from typing import Any, TypedDict, Union, Iterable
from collections import defaultdict

import numpy as np

import mlflow

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.functional import pil_to_tensor

import pyspark.sql.functions as F
from pyspark.sql import SparkSession

from PIL import Image

from tsdb.ml.utils import cut_square_detection
import tsdb.preprocessing.transformations as trf
from tsdb.preprocessing.images import make_image_metadata_udf
from tsdb.ml.infer import make_towerscout_predict_udf
from tsdb.ml.detections import YOLOv5_Detector
from tsdb.ml.efficientnet import EN_Classifier
from models.common import Detections  # Detection object for YOLOv5 model

In [0]:
class ImageMetadata(TypedDict):
    """
    A class to represent image metadata.
    height: the image height
    width: the image width
    lat: the latitude of the image
    long: the longitude of the image
    image_id: the id of the image
    map_provider: the map provider the image is from 
    image: The PIL image object
    """
    height: int
    width: int
    lat: float
    long: float
    image_id: int
    map_provider: str
    image: Image


def get_image_metadata(image_binary: bytes) -> ImageMetadata:  # pragma: no cover
        # Try to read the image and if we fail, we have to default to
        # to the null image case
        image_binary = BytesIO(image_binary)

        try:
            image = Image.open(image_binary)
            exif = image._getexif()

        except FileNotFoundError:
            exif = None
        except UnicodeDecodeError:
            exif = None

        user_comment_exif_id = 37510

        if exif is None or user_comment_exif_id not in exif:
            # we need to return with default values
            fake_image = Image.new('RGB', (640, 640), (0, 0, 0))
            return {
                "height": 640,
                "width": 640,
                "lat": 0.0,
                "long": 0.0,
                "image_id": -1,
                "map_provider": "unknown",
                "image": fake_image
            }
        
        try:
            user_comment_exif = exif[user_comment_exif_id]
            exif_dict = loads(
                user_comment_exif.decode("utf-8").replace("\'", "\"")
            )
        
        except UnicodeDecodeError as e:
            # can we gracefully handle this?
            raise ValueError(f"Unable to decode exif data: {e}")
        
        image_id = -1 if "id" not in exif_dict else int(exif_dict["id"])
        return {
            "height": image.height,
            "width": image.width,
            "lat": exif_dict["lat"],
            "long": exif_dict["lng"],
            "image_id": image_id,
            "map_provider": exif_dict["mapProvider"],
            "image": image
        }


class ImageBinaryDataset(Dataset):
    def __init__(self, images):
        self.images = images
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index) -> ImageMetadata:
         return get_image_metadata(self.images[index])
    

def inference_collate_fn(data: list[dict[str, ImageMetadata]]) -> dict[str, Union[Image,dict[str, Any]]]:
    batch = defaultdict(list)
    for item in data:
        batch["images"].append(item.pop("image"))
        batch["images_metadata"].append(item)

    return batch
    

In [0]:
request_id = "be69e91f"
user_id = "cnu4"
base_path = f"/Volumes/edav_dev_csels/towerscout/images/maps/bronze/{user_id}/{request_id}"


image_df = (
    spark
    .read
    .format("binaryFile")
    .load(base_path) # parameterize
    .select("content")
    .limit(20)
    #.repartition(8)
    #.withColumn("inference", yolo_inference_udf(F.col("content")))
)


image_df = image_df.toPandas()
image_bins = image_df["content"]

bin_dataset = ImageBinaryDataset(image_bins)

batch_size = 4
num_workers = 4

loader = DataLoader(
        bin_dataset, 
        batch_size=batch_size, 
        num_workers=num_workers,
        collate_fn=inference_collate_fn 
    )

mlflow.set_registry_uri("databricks-uc")

In [0]:
yolo_model = mlflow.pytorch.load_model(
            model_uri=f"models:/edav_dev_csels.towerscout.yolo_autoshape@aws"
        )

en_model =  mlflow.pytorch.load_model(
            model_uri=f"models:/edav_dev_csels.towerscout.efficientnet@aws"
        )


yolo_model.eval()
en_model.eval()

if torch.cuda.is_available():  # pragma: no cover
    en_model.cuda()
    yolo_model.cuda()

In [0]:
def parse_yolo_detections(
    images: list[Image],
    yolo_results: Detections,
    secondary_model: torch.nn.Module = None,
    **kwargs
) -> list[dict[str, Any]]:
    """
    A function to parse the detections from the YOLO model by converting them into a list
    of dicts with the following keys:
    - x1: the x1 coordinate of the bounding box
    - y1: the y1 coordinate of the bounding box
    - x2: the x2 coordinate of the bounding box
    - y2: the y2 coordinate of the bounding box
    - conf: the YOLO model confidence of the detection
    - class: the class of the detection
    - class_name: the name of the class of the detection
    - secondary: the secondary model confidence of the detection (if a secondary model is supplied)

    Args:
        images: the list of PIL images
        yolo_results: the Detections object from the YOLO model
        secondary_model: the secondary model used to evaluate the detections
        **kwargs: additional keyword arguments to pass to the secondary model
    Returns:
        A list of dicts with the keys from above.
    """
    parsed_results = []
    batch_detections = yolo_results.xyxyn

    for image, image_detections in zip(images, batch_detections):
        image_detections = image_detections.cpu().numpy().tolist()
        
        if secondary_model is not None:
            apply_secondary_model(secondary_model, image, image_detections, **kwargs)

        image_results = [
                    {
                        "x1": item[0],
                        "y1": item[1],
                        "x2": item[2],
                        "y2": item[3],
                        "conf": item[4],
                        "class": int(item[5]),
                        "class_name": yolo_results.names[int(item[5])],
                        "secondary": item[6] if len(item) > 6 else 1,
                    }
                    for item in image_detections
                ]

        parsed_results.append(image_results)

    return parsed_results


def apply_secondary_model(
    secondary_model: torch.nn.Module,
    image: Image,
    detections: list[np.array],
    min_conf: float = 0.25,
    max_conf: float = 0.65,
) -> None:
    """
    A function to apply the secondary model to the detections from the YOLO model. The function 
    first crops the image based on the bounding box predicted by the YOLO model, then applies 
    the secondary model to the cropped image to determine the probablity the image contains a cooling tower
    and appends the computed probability to the detection array.

    Args:
        secondary_model: the secondary model to apply to the cropped image
        image: the image to crop
        detections: list of the detections from the YOLO model for the input image
        min_conf: the minimum confidence to apply the secondary model
        max_conf: the maximum confidence to apply the secondary model
    """
    transform = transforms.Compose(
        [
            transforms.Resize([456, 456]),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=(0.5553, 0.5080, 0.4960), std=(0.1844, 0.1982, 0.2017)
            ),
        ]
    )

    for detection in detections:
        x1, y1, x2, y2, conf = detection[0:5]

        # Use secondary model only for certain confidence range
        if conf >= min_conf and conf <= max_conf:
            bbox_cropped_image = cut_square_detection(image, x1, y1, x2, y2)

            # apply transformations
            input = transform(bbox_cropped_image).unsqueeze(0)

            if torch.cuda.is_available():  # pragma: no cover
                input = input.cuda()

            # subtract from 1 because the secondary has class 0 as tower
            output = 1 - torch.sigmoid(secondary_model(input).cpu()).item()
            p2 = output
        elif conf < min_conf:
            # set secondary classifier probability to 0
            p2 = 0
        else:
            # if >= max_conf set secondary classifier probability to 1
            p2 = 1

        detection.append(p2)
    
    return 

In [0]:
from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.sql import DataFrame
import pyspark.sql.types as T
from mlflow import set_registry_uri
import pandas as pd

from tsdb.ml.utils import get_model_tags


YOLO_RETURN_TYPE = T.ArrayType(
            T.StructType([
                T.StructField("x1", T.FloatType(), True),
                T.StructField("y1", T.FloatType(), True),
                T.StructField("x2", T.FloatType(), True),
                T.StructField("y2", T.FloatType(), True),
                T.StructField("conf", T.FloatType(), True),
                T.StructField("class", T.IntegerType(), True),
                T.StructField("class_name", T.StringType(), True),
                T.StructField("secondary", T.FloatType(), True),
            ])
        )


MODEL_VERSION_STRUCT = T.StructType([
    T.StructField("yolo_model", T.StringType()),
    T.StructField("yolo_model_version", T.StringType()),
    T.StructField("efficientnet_model", T.StringType()),
    T.StructField("efficientnet_model_version", T.StringType())
])


TOWERSCOUT_IMAGE_METADATA_STRUCT = T.StructType([
        T.StructField("height", T.IntegerType()),
        T.StructField("width", T.IntegerType()),
        T.StructField("lat", T.DoubleType()),
        T.StructField("long", T.DoubleType())
    ])

In [0]:
UDF_RETURN_TYPE = T.StructType(
    [
        T.StructField("bboxes", YOLO_RETURN_TYPE),
        T.StructField("model_version", MODEL_VERSION_STRUCT),
        T.StructField("image_metadata", TOWERSCOUT_IMAGE_METADATA_STRUCT),
        T.StructField("image_id", T.IntegerType()),
        T.StructField("map_provider", T.StringType()),
    ]
)


def new_make_towerscout_predict_udf(
    catalog: str,
    schema: str,
    yolo_alias: str = "aws",
    efficientnet_alias: str = "aws",
    batch_size: int = 100,
    num_workers: int = 2,
) -> callable:
    """
    For a pandas UDF, we need the outer function to initialize the models
    and the inner function to perform the inference process. For more
    information, see the following reference by NVIDIA:
    -

    Args:
        catalog: The catalog name.
        schema: The schema name.
        yolo_alias: The alias for the YOLO model in UC.
        efficientnet_alias: The alias for the EfficientNet model in UC.
        batch_size: Batch size for the DataLoader.
        num_workers: Number of workers for the DataLoader.

    Returns:
        callable: A pandas UDF for inference.
    """
    mlflow.set_registry_uri("databricks-uc")

    yolo_model_name = f"{catalog}.{schema}.yolo_autoshape"
    en_model_name = f"{catalog}.{schema}.efficientnet"

    # Retrieves models by alias
    yolo_detector = mlflow.pytorch.load_model(
        model_uri=f"models:/{yolo_model_name}@{yolo_alias}"
    )

    en_classifier = mlflow.pytorch.load_model(
        model_uri=f"models:/{en_model_name}@{efficientnet_alias}"
    )

    yolo_detector.eval()
    en_classifier.eval()

    if torch.cuda.is_available():  # pragma: no cover
        en_classifier.cuda()
        yolo_detector.cuda()

    _, yolo_uc_version = get_model_tags(yolo_model_name, yolo_alias)
    _, en_uc_version = get_model_tags(en_model_name, efficientnet_alias)

    model_metadata = {
        "yolo_model": "yolo_autoshape",
        "yolo_model_version": yolo_uc_version,
        "efficientnet_model": "efficientnet",
        "efficientnet_model_version": en_uc_version,
    }

    @pandas_udf(UDF_RETURN_TYPE)
    @torch.no_grad()
    def predict(image_bins: pd.Series) -> pd.DataFrame:  # pragma: no cover
        """
        This predict function is distributed across executors to perform inference.

        YOLOv5 library expects the following image formats: file, URI, numpy, PIL, OpenCV, torch tensor, multiple.
        - See `Autoshape.forward()` in: https://github.com/ultralytics/yolov5/blob/master/models/common.py
        NOTE: Despite this do NOT pass tensors during inference (only training)
        - See: https://github.com/ultralytics/yolov5/issues/7030#issuecomment-1078171092

        NOTE: No need to resize images for yolov5 lib as it does it for you
        - Source: letterbox and exif_transpose funcs in:
            https://github.com/ultralytics/yolov5/blob/master/models/common.py

        Args:
            image_bins: A partition of the image binaries

        Returns:
            DataFrame: DataFrame with predicted labels and extracted image metadata.
        """
        bin_dataset = ImageBinaryDataset(image_bins)
        loader = DataLoader(
            bin_dataset,
            batch_size=batch_size,
            num_workers=num_workers,
            collate_fn=inference_collate_fn,
        )

        outputs = []

        for batch in loader:
            yolo_output = yolo_model(batch["images"])
            parsed_results = parse_yolo_detections(
                batch["images"], yolo_output, en_classifier
            )
            # try using zip instead of i
            for i in range(len(parsed_results)):
                map_provider = batch["images_metadata"][i].pop("map_provider")
                image_id = batch["images_metadata"][i].pop("image_id")
                
                outputs.append(
                    {
                        "bboxes": parsed_results[i],
                        "model_version": model_metadata,
                        "image_metadata": batch["images_metadata"][i],
                        "image_id": image_id,
                        "map_provider": map_provider,
                    }
                )

        return pd.DataFrame(outputs) 

    return predict

In [0]:
def old_make_towerscout_predict_udf(
    catalog: str,
    schema: str,
    yolo_alias: str = "aws",
    efficientnet_alias: str = "aws",
    batch_size: int = 100
) -> DataFrame:  # pragma: no cover
    """
    For a pandas UDF, we need the outer function to initialize the models
    and the inner function to perform the inference. Process. For more
    information, see the following reference by NVIDIA:
    - 

    Args:
        model_fn (InferenceModelType): The PyTorch model.
        batch_size (int): Batch size for the DataLoader.

    Returns:
        DataFrame: DataFrame with predictions.
    """ 
    set_registry_uri("databricks-uc")

    yolo_model_name = f"{catalog}.{schema}.yolo_autoshape" 
    en_model_name = f"{catalog}.{schema}.efficientnet"  

    # Retrieves models by alias and create inference objects
    yolo_detector = YOLOv5_Detector.from_uc_registry(
        model_name=yolo_model_name,
        alias=yolo_alias,
        batch_size=batch_size,
    )

    # We nearly always use efficientnet for classification but you don't have to
    en_classifier = EN_Classifier.from_uc_registry(
        model_name=en_model_name,
        alias=efficientnet_alias
    )

    metadata = {
        "yolo_model": "yolo_autoshape",
        "yolo_model_version": yolo_detector.uc_version,
        "efficientnet_model": "efficientnet",
        "efficientnet_model_version": en_classifier.uc_version,
    }

    return_type = T.StructType([
        T.StructField("bboxes", yolo_detector.return_type),
        T.StructField("model_version", MODEL_VERSION_STRUCT)
    ])

    @torch.no_grad()
    def predict(content_series_iter: Iterable[Any]):  # pragma: no cover
        """
        This predict function is distributed across executors to perform inference.

        YOLOv5 library expects the following image formats:
        For size(height=640, width=1280), RGB images example inputs are:
        #   file:        ims = 'data/images/zidane.jpg'  # str or PosixPath
        #   URI:             = 'https://ultralytics.com/images/zidane.jpg'
        #   OpenCV:          = cv2.imread('image.jpg')[:,:,::-1]  # HWC BGR to RGB x(640,1280,3)
        #   PIL:             = Image.open('image.jpg') or ImageGrab.grab()  # HWC x(640,1280,3)
        #   numpy:           = np.zeros((640,1280,3))  # HWC
        #   torch:           = torch.zeros(16,3,320,640)  # BCHW (scaled to size=640, 0-1 values)
        #   multiple:        = [Image.open('image1.jpg'), Image.open('image2.jpg'), ...]  # list of images
        - Source: https://github.com/ultralytics/yolov5/blob/master/models/common.py
        
        The ultralytics lib accepts the following image formats:
        - Source: https://github.com/ultralytics/ultralytics/blob/main/ultralytics/engine/model.py 
        

        # No need to resize for yolov5 lib as it does it for you 
        - Source: letterbox and exif_transpose funcs in:
            https://github.com/ultralytics/yolov5/blob/master/models/common.py

        Args:
            content_series_iter: Iterator over content series.

        Yields:
            DataFrame: DataFrame with predicted labels.
        """
        for content_series in content_series_iter:
            # Create dataset object to apply transformations
            image_batch = [
                Image.open(BytesIO(content)).convert("RGB")
                for content in content_series
            ]

            # Perform inference on batch
            outputs = yolo_detector.predict(
                model_input=image_batch, 
                secondary=en_classifier
            )

            outputs = [
                {"bboxes": output, "model_version": metadata}
                for output in outputs
            ]
            yield pd.DataFrame(outputs)


    return pandas_udf(return_type, PandasUDFType.SCALAR_ITER)(predict)


def old_make_image_metadata_udf(spark: SparkSession):  # pragma: no cover
    towerscout_image_metadata_schema = T.StructType([
        T.StructField("height", T.IntegerType()),
        T.StructField("width", T.IntegerType()),
        T.StructField("lat", T.DoubleType()),
        T.StructField("long", T.DoubleType()),
        T.StructField("image_id", T.IntegerType()),
        T.StructField("map_provider", T.StringType())
    ])

    def get_image_metadata(image_binary: T.BinaryType):  # pragma: no cover
        image = Image.open(BytesIO(image_binary))
        exif = image._getexif()
        user_comment_exif_id = 37510

        if exif is None or user_comment_exif_id not in exif:
            # we need to return with default values
            return {
                "height": image.height,
                "width": image.width,
                "lat": 0.0,
                "long": 0.0,
                "id": -1,
                "map_provider": "unknown"
            }
        
        try:
            user_comment_exif = exif[user_comment_exif_id]
            exif_dict = loads(
                user_comment_exif.decode("utf-8").replace("\'", "\"")
            )
        
        except UnicodeDecodeError as e:
            raise ValueError(f"Unable to decode exif data: {e}")
        
        image_id = -1 if "id" not in exif_dict else int(exif_dict["id"])
        return {
            "height": image.height,
            "width": image.width,
            "lat": exif_dict["lat"],
            "long": exif_dict["lng"],
            "image_id": image_id,
            "map_provider": exif_dict["mapProvider"]
        }
    
    return spark.udf.register(
        "get_image_metadata_udf",
        get_image_metadata, 
        towerscout_image_metadata_schema
    )

In [0]:
catalog, schema = "edav_dev_csels", "towerscout"
batch_size = 8
num_workers = 20
num_partitions = 6

new_towerscout_inference_udf = new_make_towerscout_predict_udf(catalog, schema, yolo_alias="aws", efficientnet_alias="aws", batch_size=batch_size, num_workers=num_workers)

old_towerscout_inference_udf = old_make_towerscout_predict_udf(catalog, schema, batch_size=batch_size)
old_image_metadata_udf = old_make_image_metadata_udf(spark)

In [0]:
#base_path = f"/Volumes/{catalog}/{schema}/images/maps/bronze/{user_id}/*"
base_path = f"/Volumes/{catalog}/{schema}/images/maps/bronze/{user_id}/{request_id}"

image_df = (
    spark
    .read
    .format("binaryFile")
    .load(base_path) # parameterize
    .select("content", "path")
    .repartition(num_partitions)
    #.limit(200)
)

print(f"Number of images: {image_df.count()}")
print(f"Number of partitions: {image_df.rdd.getNumPartitions()}")

In [0]:
transformed_df_old = (
    image_df
    .transform(trf.perform_inference, old_towerscout_inference_udf)
    .transform(trf.extract_metadata, old_image_metadata_udf)
)

display(transformed_df_old)

In [0]:
transformed_df_new = (
    image_df
    .transform(trf.perform_inference, new_towerscout_inference_udf)
)

display(transformed_df_new)

In [0]:
# print(dir(towerscout_inference_udf))
# print(hasattr(towerscout_inference_udf, "returnType"))
# print(towerscout_inference_udf.returnType == UDF_RETURN_TYPE)

# print(isinstance(towerscout_inference_udf, Callable))